In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import dataset, Reader
from surprise.prediction_algorithms import KNNBasic
from surprise.model_selection import train_test_split

In [2]:
DATA = Path('marketing')

In [3]:
market_df = pd.read_csv(DATA / 'marketing.csv', 
                        names= ['Income', 'Sex', 'MaritalStatus', 'Age', 'Education', 'Occupation', 'YearsInBayAreas', 'DualIncome', 'NumberInHoushold', 'NumberOfChildren',
                                'HousholderStatus', 'TypeOfHome', 'EthnicClassification', 'LanguageInHome'])
market_df.head(5)

,Income,Sex,MaritalStatus,Age,Education,Occupation,YearsInBayAreas,DualIncome,NumberInHoushold,NumberOfChildren,HousholderStatus,TypeOfHome,EthnicClassification,LanguageInHome
0,9,2,1.0,5,4.0,5.0,5.0,3,3.0,0,1.0,1.0,7.0,NaN
1,9,1,1.0,5,5.0,5.0,5.0,3,5.0,2,1.0,1.0,7.0,1.0
2,9,2,1.0,3,5.0,1.0,5.0,2,3.0,1,2.0,3.0,7.0,1.0
3,1,2,5.0,1,2.0,6.0,5.0,1,4.0,2,3.0,1.0,7.0,1.0
4,1,2,5.0,1,2.0,6.0,3.0,1,4.0,2,3.0,1.0,7.0,1.0


In [4]:
market_df = market_df.dropna()
market_df.shape

(6876, 14)

In [5]:
#bin the ordinal data 
market_df_binned = market_df
market_df_binned['Age'] = pd.cut(market_df['Age'], bins= [0,4,7], labels=['AgeBlw45', 'AgeAbv45'])
market_df_binned['Education'] = pd.cut(market_df['Education'], bins= [0,3,6], labels=['HighSchool', 'College'])
market_df_binned['Income'] = pd.cut(market_df['Income'], bins= [0,5,9], labels=['IncBlw30K', 'IncAbv30K'])
market_df_binned['YearsInBayAreas'] = pd.cut(market_df['YearsInBayAreas'], bins= [0,3,5], labels=['YrsBlw7', 'YrsAbv7'])
market_df_binned['NumberInHoushold'] = pd.cut(market_df['NumberInHoushold'], bins= [0,5,9], labels=['NumHousBlw6', 'NumHousAbv6'])
market_df_binned['NumberOfChildren'] = pd.cut(market_df['NumberOfChildren'], bins= [-1,5,9], labels=['NumChlBlw6', 'NumChlAbv6'])

In [6]:
market_df_binned

,Income,Sex,MaritalStatus,Age,Education,Occupation,YearsInBayAreas,DualIncome,NumberInHoushold,NumberOfChildren,HousholderStatus,TypeOfHome,EthnicClassification,LanguageInHome
1,IncAbv30K,1,1.0,AgeAbv45,College,5.0,YrsAbv7,3,NumHousBlw6,NumChlBlw6,1.0,1.0,7.0,1.0
2,IncAbv30K,2,1.0,AgeBlw45,College,1.0,YrsAbv7,2,NumHousBlw6,NumChlBlw6,2.0,3.0,7.0,1.0
3,IncBlw30K,2,5.0,AgeBlw45,HighSchool,6.0,YrsAbv7,1,NumHousBlw6,NumChlBlw6,3.0,1.0,7.0,1.0
4,IncBlw30K,2,5.0,AgeBlw45,HighSchool,6.0,YrsBlw7,1,NumHousBlw6,NumChlBlw6,3.0,1.0,7.0,1.0
5,IncAbv30K,1,1.0,AgeAbv45,College,8.0,YrsAbv7,3,NumHousBlw6,NumChlBlw6,1.0,1.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8988,IncBlw30K,2,5.0,AgeBlw45,HighSchool,2.0,YrsAbv7,1,NumHousBlw6,NumChlBlw6,3.0,1.0,7.0,1.0
8989,IncBlw30K,1,5.0,AgeBlw45,College,1.0,YrsAbv7,1,NumHousBlw6,NumChlBlw6,3.0,1.0,7.0,1.0
8990,IncBlw30K,2,5.0,AgeBlw45,HighSchool,1.0,YrsAbv7,1,NumHousBlw6,NumChlBlw6,3.0,1.0,7.0,1.0
8991,IncBlw30K,1,1.0,AgeAbv45,College,3.0,YrsAbv7,2,NumHousBlw6,NumChlBlw6,2.0,3.0,7.0,1.0


In [7]:
#make keys for categorical data so dummies are more informative
key_Sex = {1:'Male', 2:'Female'}
key_MaritalStatus = {1: 'Married', 2: 'LivingTogetherUnmarried', 3: "Divorced/Seperated", 4: 'Widowed', 5:'Single/NeverMarried'}
key_Occupation = {1: 'Professional/Managerial', 2:'SalesWorker', 3:'Factory Worker/Laborer/Driver', 4: 'Clerical/Service Worker', 5:'Homemaker', 6:'Student', 7:'Military', 8:'Retired', 9:'Unemployed'}
key_DualIncome = {1:"Unmarried", 2:"Yes", 3:'No'}
key_HousholderStatus = {1:'Own', 2:'Rent', 3:'LiveWithFamily'}
key_TypeOfHome = {1:'House', 2:'Condominium', 3:'Apartment', 4:'MobileHome', 5:'Other'}
key_EthnicClassification = {1:'AmericanIndian', 2:'Asian', 3:'Black', 4:'EastIndian', 5:'Hispanic', 6:'PacificIslander', 7:'White', 8:'Other'}
key_LanguageInHome = {1:'English', 2:'Spanish', 3:'Other'}

In [8]:
market_df_binned['Sex']= market_df_binned['Sex'].replace(key_Sex)
market_df_binned['MaritalStatus']= market_df_binned['MaritalStatus'].replace(key_MaritalStatus)
market_df_binned['Occupation']= market_df_binned['Occupation'].replace(key_Occupation)
market_df_binned['DualIncome']= market_df_binned['DualIncome'].replace(key_DualIncome)
market_df_binned['HousholderStatus']= market_df_binned['HousholderStatus'].replace(key_HousholderStatus)
market_df_binned['TypeOfHome']= market_df_binned['TypeOfHome'].replace(key_TypeOfHome)
market_df_binned['EthnicClassification']= market_df_binned['EthnicClassification'].replace(key_EthnicClassification)
market_df_binned['LanguageInHome']= market_df_binned['LanguageInHome'].replace(key_LanguageInHome)
market_df_binned

,Income,Sex,MaritalStatus,Age,Education,Occupation,YearsInBayAreas,DualIncome,NumberInHoushold,NumberOfChildren,HousholderStatus,TypeOfHome,EthnicClassification,LanguageInHome
1,IncAbv30K,Male,Married,AgeAbv45,College,Homemaker,YrsAbv7,No,NumHousBlw6,NumChlBlw6,Own,House,White,English
2,IncAbv30K,Female,Married,AgeBlw45,College,Professional/Managerial,YrsAbv7,Yes,NumHousBlw6,NumChlBlw6,Rent,Apartment,White,English
3,IncBlw30K,Female,Single/NeverMarried,AgeBlw45,HighSchool,Student,YrsAbv7,Unmarried,NumHousBlw6,NumChlBlw6,LiveWithFamily,House,White,English
4,IncBlw30K,Female,Single/NeverMarried,AgeBlw45,HighSchool,Student,YrsBlw7,Unmarried,NumHousBlw6,NumChlBlw6,LiveWithFamily,House,White,English
5,IncAbv30K,Male,Married,AgeAbv45,College,Retired,YrsAbv7,No,NumHousBlw6,NumChlBlw6,Own,House,White,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8988,IncBlw30K,Female,Single/NeverMarried,AgeBlw45,HighSchool,SalesWorker,YrsAbv7,Unmarried,NumHousBlw6,NumChlBlw6,LiveWithFamily,House,White,English
8989,IncBlw30K,Male,Single/NeverMarried,AgeBlw45,College,Professional/Managerial,YrsAbv7,Unmarried,NumHousBlw6,NumChlBlw6,LiveWithFamily,House,White,English
8990,IncBlw30K,Female,Single/NeverMarried,AgeBlw45,HighSchool,Professional/Managerial,YrsAbv7,Unmarried,NumHousBlw6,NumChlBlw6,LiveWithFamily,House,White,English
8991,IncBlw30K,Male,Married,AgeAbv45,College,Factory Worker/Laborer/Driver,YrsAbv7,Yes,NumHousBlw6,NumChlBlw6,Rent,Apartment,White,English


In [9]:
#take fully binned and categorized values and turn into dummies 
market_df_Dummied = pd.get_dummies(market_df_binned,prefix_sep='_', drop_first=False)
market_df_Dummied.head(3)

,Income_IncBlw30K,Income_IncAbv30K,Sex_Female,Sex_Male,MaritalStatus_Divorced/Seperated,MaritalStatus_LivingTogetherUnmarried,MaritalStatus_Married,MaritalStatus_Single/NeverMarried,MaritalStatus_Widowed,Age_AgeBlw45,...,EthnicClassification_Asian,EthnicClassification_Black,EthnicClassification_EastIndian,EthnicClassification_Hispanic,EthnicClassification_Other,EthnicClassification_PacificIslander,EthnicClassification_White,LanguageInHome_English,LanguageInHome_Other,LanguageInHome_Spanish
1,False,True,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,True,False,False
2,False,True,True,False,False,False,True,False,False,True,...,False,False,False,False,False,False,True,True,False,False
3,True,False,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,True,True,False,False


In [10]:
market_df_Dummied.columns

Index(['Income_IncBlw30K', 'Income_IncAbv30K', 'Sex_Female', 'Sex_Male',
       'MaritalStatus_Divorced/Seperated',
       'MaritalStatus_LivingTogetherUnmarried', 'MaritalStatus_Married',
       'MaritalStatus_Single/NeverMarried', 'MaritalStatus_Widowed',
       'Age_AgeBlw45', 'Age_AgeAbv45', 'Education_HighSchool',
       'Education_College', 'Occupation_Clerical/Service Worker',
       'Occupation_Factory Worker/Laborer/Driver', 'Occupation_Homemaker',
       'Occupation_Military', 'Occupation_Professional/Managerial',
       'Occupation_Retired', 'Occupation_SalesWorker', 'Occupation_Student',
       'Occupation_Unemployed', 'YearsInBayAreas_YrsBlw7',
       'YearsInBayAreas_YrsAbv7', 'DualIncome_No', 'DualIncome_Unmarried',
       'DualIncome_Yes', 'NumberInHoushold_NumHousBlw6',
       'NumberInHoushold_NumHousAbv6', 'NumberOfChildren_NumChlBlw6',
       'NumberOfChildren_NumChlAbv6', 'HousholderStatus_LiveWithFamily',
       'HousholderStatus_Own', 'HousholderStatus_Rent', 'Ty

In [11]:
# create frequent itemsets
itemsets = apriori(market_df_Dummied, min_support=0.1, use_colnames=True)
itemsets

,support,itemsets
0,0.499418,(Income_IncBlw30K)
1,0.500582,(Income_IncAbv30K)
2,0.553956,(Sex_Female)
3,0.446044,(Sex_Male)
4,0.385689,(MaritalStatus_Married)
...,...,...
10801,0.100785,"(NumberOfChildren_NumChlBlw6, Education_Colleg..."
10802,0.141507,"(NumberOfChildren_NumChlBlw6, Education_Colleg..."
10803,0.127400,"(NumberOfChildren_NumChlBlw6, Education_Colleg..."
10804,0.103403,"(NumberOfChildren_NumChlBlw6, YearsInBayAreas_..."


In [26]:
# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.8)
rules_sorted= rules.sort_values(by=['lift'], ascending=False)
pd.set_option('display.max_colwidth', None)

In [27]:
rules_sorted.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
33784,"(TypeOfHome_House, Education_HighSchool, MaritalStatus_Single/NeverMarried)","(HousholderStatus_LiveWithFamily, DualIncome_Unmarried, Age_AgeBlw45)",0.129290,0.197062,0.108202,0.836895,4.246858,0.082724,4.922839,0.878056
51702,"(NumberOfChildren_NumChlBlw6, TypeOfHome_House, Education_HighSchool, MaritalStatus_Single/NeverMarried)","(HousholderStatus_LiveWithFamily, DualIncome_Unmarried, Age_AgeBlw45)",0.127691,0.197062,0.106603,0.834852,4.236488,0.081440,4.861927,0.875785
51709,"(TypeOfHome_House, Education_HighSchool, MaritalStatus_Single/NeverMarried)","(NumberOfChildren_NumChlBlw6, HousholderStatus_LiveWithFamily, DualIncome_Unmarried, Age_AgeBlw45)",0.129290,0.194735,0.106603,0.824522,4.234065,0.081425,4.588976,0.877239
33781,"(TypeOfHome_House, Education_HighSchool, MaritalStatus_Single/NeverMarried, Age_AgeBlw45)","(HousholderStatus_LiveWithFamily, DualIncome_Unmarried)",0.128272,0.199825,0.108202,0.843537,4.221371,0.082570,5.114159,0.875400
51695,"(NumberOfChildren_NumChlBlw6, MaritalStatus_Single/NeverMarried, TypeOfHome_House, Education_HighSchool, Age_AgeBlw45)","(HousholderStatus_LiveWithFamily, DualIncome_Unmarried)",0.126672,0.199825,0.106603,0.841561,4.211482,0.081290,5.050377,0.873159


from the limited window we can see above we can see a Strong relationship between Living in a house with a high school level of education and living with familiy and being unmarried. This doesnt give us alot of data, lets look at a subset that has shown buying power, a homeowner, next. 

In [28]:
homeOwner_df = market_df_Dummied[market_df_Dummied['HousholderStatus_Own']==1]
homeOwner_df.head(3)

,Income_IncBlw30K,Income_IncAbv30K,Sex_Female,Sex_Male,MaritalStatus_Divorced/Seperated,MaritalStatus_LivingTogetherUnmarried,MaritalStatus_Married,MaritalStatus_Single/NeverMarried,MaritalStatus_Widowed,Age_AgeBlw45,...,EthnicClassification_Asian,EthnicClassification_Black,EthnicClassification_EastIndian,EthnicClassification_Hispanic,EthnicClassification_Other,EthnicClassification_PacificIslander,EthnicClassification_White,LanguageInHome_English,LanguageInHome_Other,LanguageInHome_Spanish
1,False,True,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,True,False,False
5,False,True,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,True,False,False
11,True,False,True,False,False,True,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False


In [30]:
itemsets_homeowner = apriori(homeOwner_df, min_support=0.1, use_colnames=True)
rules_homeowner = association_rules(itemsets_homeowner, metric='confidence', min_threshold=0.8)
rules_homeowner_sorted= rules_homeowner.sort_values(by=['lift'], ascending=False)
rules_homeowner_sorted.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
28618,"(HousholderStatus_Own, MaritalStatus_Single/NeverMarried)","(NumberOfChildren_NumChlBlw6, DualIncome_Unmarried, Age_AgeBlw45)",0.118034,0.179954,0.104876,0.888525,4.937522,0.083636,7.356299,0.904195
28620,(MaritalStatus_Single/NeverMarried),"(NumberOfChildren_NumChlBlw6, HousholderStatus_Own, DualIncome_Unmarried, Age_AgeBlw45)",0.118034,0.179954,0.104876,0.888525,4.937522,0.083636,7.356299,0.904195
6567,(MaritalStatus_Single/NeverMarried),"(NumberOfChildren_NumChlBlw6, Age_AgeBlw45, DualIncome_Unmarried)",0.118034,0.179954,0.104876,0.888525,4.937522,0.083636,7.356299,0.904195
6571,"(HousholderStatus_Own, MaritalStatus_Single/NeverMarried)","(Age_AgeBlw45, DualIncome_Unmarried)",0.118034,0.181115,0.105263,0.891803,4.923974,0.083885,7.568487,0.903563
877,(MaritalStatus_Single/NeverMarried),"(Age_AgeBlw45, DualIncome_Unmarried)",0.118034,0.181115,0.105263,0.891803,4.923974,0.083885,7.568487,0.903563
6574,(MaritalStatus_Single/NeverMarried),"(HousholderStatus_Own, Age_AgeBlw45, DualIncome_Unmarried)",0.118034,0.181115,0.105263,0.891803,4.923974,0.083885,7.568487,0.903563
28616,"(NumberOfChildren_NumChlBlw6, MaritalStatus_Single/NeverMarried)","(HousholderStatus_Own, DualIncome_Unmarried, Age_AgeBlw45)",0.117647,0.181115,0.104876,0.891447,4.922009,0.083569,7.543672,0.903075
28611,"(NumberOfChildren_NumChlBlw6, HousholderStatus_Own, MaritalStatus_Single/NeverMarried)","(DualIncome_Unmarried, Age_AgeBlw45)",0.117647,0.181115,0.104876,0.891447,4.922009,0.083569,7.543672,0.903075
6564,"(NumberOfChildren_NumChlBlw6, MaritalStatus_Single/NeverMarried)","(Age_AgeBlw45, DualIncome_Unmarried)",0.117647,0.181115,0.104876,0.891447,4.922009,0.083569,7.543672,0.903075
6586,"(NumberInHoushold_NumHousBlw6, MaritalStatus_Single/NeverMarried)","(NumberOfChildren_NumChlBlw6, DualIncome_Unmarried)",0.108746,0.302245,0.108746,1.000000,3.308579,0.075878,inf,0.782892


SO here we can see that a unmarried homeowner is more likely to be under the age of 45 and have no children so we can exclude them from certain market groups when advertising to them. Lets see if rentors reveal anything

In [31]:
renter_df = market_df_Dummied[market_df_Dummied['HousholderStatus_Rent']==1]
itemsets_renter = apriori(renter_df, min_support=0.1, use_colnames=True)
rules_renter = association_rules(itemsets_renter, metric='confidence', min_threshold=0.8)
rules_renter_sorted= rules_renter.sort_values(by=['lift'], ascending=False)
rules_renter_sorted.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
70068,"(NumberOfChildren_NumChlBlw6, EthnicClassification_White, DualIncome_Yes)","(HousholderStatus_Rent, NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
16515,"(EthnicClassification_White, DualIncome_Yes)","(NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
70072,"(EthnicClassification_White, DualIncome_Yes)","(HousholderStatus_Rent, NumberOfChildren_NumChlBlw6, MaritalStatus_Married, NumberInHoushold_NumHousBlw6, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
40270,"(NumberOfChildren_NumChlBlw6, EthnicClassification_White, DualIncome_Yes)","(NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
40274,"(EthnicClassification_White, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
70067,"(HousholderStatus_Rent, EthnicClassification_White, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
40285,"(HousholderStatus_Rent, EthnicClassification_White, DualIncome_Yes)","(NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
40289,"(EthnicClassification_White, DualIncome_Yes)","(HousholderStatus_Rent, NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
70057,"(HousholderStatus_Rent, DualIncome_Yes, EthnicClassification_White, NumberOfChildren_NumChlBlw6)","(NumberInHoushold_NumHousBlw6, MaritalStatus_Married, LanguageInHome_English)",0.138793,0.241499,0.119015,0.857500,3.550740,0.085496,5.322814,0.834141
59810,"(NumberOfChildren_NumChlBlw6, MaritalStatus_Married, Income_IncAbv30K, Age_AgeBlw45, LanguageInHome_English)","(HousholderStatus_Rent, DualIncome_Yes)",0.124913,0.230743,0.101319,0.811111,3.515221,0.072496,4.072539,0.817659


Ok so looking at the subset of data for renters we can see that married white couples with less than 6 kids and dual incomes are more likley to rent. We can use that to possibly advertise products that are more geared towards couples such as gifts or vacations. Ok so next lets go back to our original dataset and mess with the support and confidence levels a bit

In [37]:
itemsets = apriori(market_df_Dummied, min_support=0.2, use_colnames=True)
rules = association_rules(itemsets, metric='confidence', min_threshold=0.8)
rules_sorted= rules.sort_values(by=['lift'], ascending=False)
rules_sorted.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3648,"(NumberInHoushold_NumHousBlw6, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, MaritalStatus_Married, LanguageInHome_English)",0.243601,0.354712,0.211751,0.869254,2.450590,0.125343,4.935422,0.782570
3641,"(NumberOfChildren_NumChlBlw6, NumberInHoushold_NumHousBlw6, DualIncome_Yes)","(MaritalStatus_Married, LanguageInHome_English)",0.243601,0.355148,0.211751,0.869254,2.447579,0.125237,4.932085,0.781906
1469,"(NumberInHoushold_NumHousBlw6, DualIncome_Yes)","(MaritalStatus_Married, LanguageInHome_English)",0.243601,0.355148,0.211751,0.869254,2.447579,0.125237,4.932085,0.781906
3642,"(NumberOfChildren_NumChlBlw6, LanguageInHome_English, DualIncome_Yes)","(NumberInHoushold_NumHousBlw6, MaritalStatus_Married)",0.233711,0.371001,0.211751,0.906036,2.442142,0.125044,6.694053,0.770628
3649,"(LanguageInHome_English, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, NumberInHoushold_NumHousBlw6, MaritalStatus_Married)",0.233857,0.371001,0.211751,0.905473,2.440623,0.124990,6.654152,0.770442


So this new set of parameters shows us that a dual income houshold with less than 6 people has less than six kids and has people that are married. Not useful, less try adjusting the Minimum confidenece level instead of the minimum support. 

In [38]:
itemsets = apriori(market_df_Dummied, min_support=0.1, use_colnames=True)
rules = association_rules(itemsets, metric='confidence', min_threshold=0.9)
rules_sorted= rules.sort_values(by=['lift'], ascending=False)
rules_sorted.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
30814,"(EthnicClassification_White, HousholderStatus_Own, NumberInHoushold_NumHousBlw6, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, Income_IncAbv30K, MaritalStatus_Married, LanguageInHome_English)",0.111111,0.280396,0.101076,0.909686,3.244295,0.069921,7.967794,0.778237
30805,"(NumberOfChildren_NumChlBlw6, DualIncome_Yes, EthnicClassification_White, HousholderStatus_Own, NumberInHoushold_NumHousBlw6)","(Income_IncAbv30K, MaritalStatus_Married, LanguageInHome_English)",0.111111,0.280541,0.101076,0.909686,3.242613,0.069905,7.966184,0.778058
25112,"(EthnicClassification_White, HousholderStatus_Own, NumberInHoushold_NumHousBlw6, DualIncome_Yes)","(Income_IncAbv30K, MaritalStatus_Married, LanguageInHome_English)",0.111111,0.280541,0.101076,0.909686,3.242613,0.069905,7.966184,0.778058
25148,"(EthnicClassification_White, HousholderStatus_Own, DualIncome_Yes)","(NumberOfChildren_NumChlBlw6, Income_IncAbv30K, MaritalStatus_Married, LanguageInHome_English)",0.113583,0.280396,0.103112,0.907810,3.237606,0.071264,7.805709,0.779690
15061,"(EthnicClassification_White, HousholderStatus_Own, DualIncome_Yes)","(Income_IncAbv30K, MaritalStatus_Married, LanguageInHome_English)",0.113583,0.280541,0.103112,0.907810,3.235928,0.071247,7.804132,0.779509


Ok so this is slightly better info, We can see that a married white individual that owns their dual income household will primarily speak english and have an income above $30,000. If nothing else this can help us not send them services for english as a second language household and advertise more expensive products towards them for their economic bracket. The fatc that they own the house also opens up the posibility to sell them services related to that such as insurance, renovations, repair, selling, etc. 
There also apears to be a sense that they will have less than 6 children. Which might affect if that household will be receptive to childcare or extracurricular activities offered by us. 